<a href="https://colab.research.google.com/github/CT608/Kaggle_Project/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# package


In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer
import pickle
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

# import data

In [5]:
sessions2 = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/sessions2.csv")
users = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/users.csv")

users.drop(columns=["Unnamed: 0"], inplace=True)
sessions2.drop(columns=["Unnamed: 0"], inplace=True)

x = pd.merge(users, sessions2, how="left", on="id")

In [6]:
train_users = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/train_users_2.csv")
test_users = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/test_users.csv")

In [7]:
sessions = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/sessions.csv")

In [8]:
x_train = x[x["mark"] == 0].drop(columns=["mark"])
x_test = x[x["mark"] == 1].drop(columns=["mark"])

In [9]:
x_train["country_destination"] = train_users["country_destination"]

In [10]:
y_train = x_train["country_destination"]

In [11]:
x_train.drop(columns=["date_first_booking", "country_destination", "id"], inplace=True)
x_test.drop(columns=["date_first_booking", "id"], inplace=True)

In [12]:
x_train.shape

(213451, 698)

In [13]:
y_train = pd.DataFrame(y_train)
y_encoder = OrdinalEncoder()
y_train = y_encoder.fit_transform(y_train)

In [14]:
y_encoder.categories_

[array(['AU', 'CA', 'DE', 'ES', 'FR', 'GB', 'IT', 'NDF', 'NL', 'PT', 'US',
        'other'], dtype=object)]

# define score

In [94]:
def dcg_score(y_true, y_score, k=5):
    
    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """
    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    
    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """
    
    lb = LabelBinarizer()
    lb.fit(range(predictions.shape[1] + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)

# model


## action model

In [154]:
action_col = x_train.columns[161:684]

In [155]:
x_train_act = x_train[action_col]
x_test_act = x_test[action_col]

In [ ]:
gb_action = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mlogloss', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, n_estimators=100, n_jobs=-1, objective='multi:softmax', random_state=0,
       reg_lambda=1, subsample=1)

In [ ]:
gb_action.fit(x_train_act,y_train)
pickle.dump(gb_action, open("/content/drive/MyDrive/Project/Airbnb New User Bookings/gb_action.pickle.dat", "wb"))

In [152]:
gb_action = pickle.load(open("/content/drive/MyDrive/Project/Airbnb New User Bookings/gb_action.pickle.dat", "rb"))

In [156]:
y_tr_pred_act = gb_action.predict_proba(x_train_act)
y_te_pred_act = gb_action.predict_proba(x_test_act)

In [157]:
pd.DataFrame(y_tr_pred_act).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_act.csv")
pd.DataFrame(y_te_pred_act).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_act.csv")

In [159]:
ndcg_score(y_train, y_tr_pred_act, k=5)

0.816572920215975

# other model

In [ ]:
other_col = np.array(x_train.columns[12:160])
other_col = np.append(other_col, x_train.columns[684:])

In [ ]:
x_train_othr = x_train[other_col]
x_test_othr = x_test[other_col]

In [ ]:
y_train = pd.DataFrame(y_train)

## parameter optimization

In [ ]:
params = {
    'num_leaves': [200, 150, 180],
    'num_iterations': [80, 50], 
    "learning_rate": [0.01],
    "max_depth": [3, 4, 5],
          }

In [ ]:
lgb_othr = lgb.LGBMClassifier(objective="multiclass", num_classes=12)

search_model = RandomizedSearchCV(
    lgb_othr,
    params, 
    verbose = 100, 
    cv = 3, 
    n_jobs = -1, 
    scoring = ndcg_scorer,
    return_train_score=True
)

search_model.fit(x_train_othr, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=LGBMClassifier(num_classes=12,
                                            objective='multiclass'),
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01],
                                        'max_depth': [3, 4, 5],
                                        'num_iterations': [80, 50],
                                        'num_leaves': [200, 150, 180]},
                   return_train_score=True,
                   scoring=make_scorer(ndcg_score, needs_proba=True, k=5),
                   verbose=100)

In [ ]:
search_model.best_params_

{'learning_rate': 0.01,
 'max_depth': 5,
 'num_iterations': 50,
 'num_leaves': 200}

In [ ]:
search_model.best_score_

0.8067732215845593

In [ ]:
ndcg_score(y_train, aaa, k=5)

0.8144566093617223

## predict


In [55]:
lgb_othr = lgb.LGBMClassifier(
    objective="multiclass", 
    num_classes=12,
    learning_rate = 0.01,
    max_depth = 5,
    num_iterations = 50,
    num_leaves = 200,
    )

In [128]:
def get_oof(clf, x_train_f, y_train_f, x_test_f, nfolds):
    ntrain = x_train_f.shape[0]
    ntest = x_test_f.shape[0]
    nclass = len(np.unique(y_train_f))

    oof_train = np.zeros((ntrain, nclass))
    oof_test = np.zeros((ntest, nclass))

    kf = KFold(n_splits=nfolds, shuffle=True, random_state=0)

    for train_index, valid_index in kf.split(x_train_f):
        x_tr = x_train_f.iloc[train_index]
        y_tr = y_train_f[train_index]
        x_vl = x_train_f.iloc[valid_index]

        clf.fit(x_tr, y_tr)

        oof_train[valid_index] = clf.predict_proba(x_vl)
        oof_test += clf.predict_proba(x_test_f)

    oof_test = oof_test / nfolds
    return oof_train, oof_test

In [149]:
y_tr_pred_othr, y_te_pred_othr = get_oof(lgb_othr, x_train_othr, y_train, x_test_othr, 3)

In [150]:
pd.DataFrame(y_tr_pred_othr).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_othr.csv")
pd.DataFrame(y_te_pred_othr).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_othr.csv")

In [151]:
ndcg_score(y_train, y_tr_pred_othr, k=5)

0.8067599694970453

# age and gender model

In [138]:
ag_col = x_train.columns[0:11]
x_train_ag = x_train[ag_col]
x_test_ag = x_test[ag_col]

## parameter optimation

In [139]:
params = {
    'num_leaves': [200, 150, 180],
    'num_iterations': [80, 50], 
    "learning_rate": [0.01],
    "max_depth": [3, 4, 5],
          }

In [141]:
lgb_ag = lgb.LGBMClassifier(objective="multiclass", num_classes=12)

search_model = RandomizedSearchCV(
    lgb_ag,
    params, 
    verbose = 10, 
    cv = 3, 
    n_jobs = -1, 
    scoring = ndcg_scorer,
    return_train_score=True
)

search_model.fit(x_train_ag, y_train)
search_model.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'learning_rate': 0.01,
 'max_depth': 3,
 'num_iterations': 50,
 'num_leaves': 200}

In [142]:
search_model.best_score_

0.7972958870386279

In [144]:
lgb_ag = lgb.LGBMClassifier(
    objective="multiclass", 
    num_classes=12,
    learning_rate = 0.01,
    max_depth = 3,
    num_iterations = 50,
    num_leaves = 200,
    )

In [146]:
y_tr_pred_ag, y_te_pred_ag = get_oof(lgb_ag, x_train_ag, y_train, x_test_ag, 3)

In [147]:
pd.DataFrame(y_tr_pred_ag).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_ag.csv")
pd.DataFrame(y_te_pred_ag).to_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_ag.csv")

In [148]:
ndcg_score(y_train, y_tr_pred_ag, k=5)

0.8067998771231959

# multi

In [73]:
y_tr_pred_act = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_act.csv", )
y_te_pred_act = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_act.csv")

y_tr_pred_othr = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_othr.csv")
y_te_pred_othr = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_othr.csv")

y_tr_pred_ag = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_tr_pred_ag.csv")
y_te_pred_ag = pd.read_csv("/content/drive/MyDrive/Project/Airbnb New User Bookings/y_te_pred_ag.csv")

y_tr_pred_act.drop(columns=["Unnamed: 0"], inplace=True)
y_te_pred_act.drop(columns=["Unnamed: 0"], inplace=True)
y_tr_pred_othr.drop(columns=["Unnamed: 0"], inplace=True)
y_te_pred_othr.drop(columns=["Unnamed: 0"], inplace=True)
y_tr_pred_ag.drop(columns=["Unnamed: 0"], inplace=True)
y_te_pred_ag.drop(columns=["Unnamed: 0"], inplace=True)

In [74]:
names = np.array(["act", "othr", "ag"])

In [75]:
col = np.array([])

In [76]:
for name in names:
    for i in np.arange(12):
        name_i = name +str(i)
        col = np.append(col, name_i)

In [86]:
x_tr_lay1 = pd.concat([y_tr_pred_act, y_tr_pred_othr, y_tr_pred_ag], axis=1)
x_tr_lay1.columns = col

x_te_lay1 = pd.concat([y_te_pred_act, y_te_pred_othr, y_te_pred_ag], axis=1)
x_te_lay1.columns = col

In [87]:
lg = LogisticRegression()
lg.fit(x_tr_lay1, y_train)

LogisticRegression()

In [90]:
y_tr_pred  = lg.predict_proba(x_tr_lay1)

In [95]:
ndcg_score(y_train, y_tr_pred, k=5)

0.8237972038937806

In [96]:
test_id = test_users['id'].values

In [97]:
pred_y = lg.predict_proba(x_te_lay1)

In [98]:
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(test_id)):
    idx = test_id[i]
    ids += [idx] * 5
    cts += y_encoder.inverse_transform(np.argsort(pred_y[i])[::-1].reshape(-1, 1))[:5].tolist()
# Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('/content/drive/MyDrive/Project/Airbnb New User Bookings/sub.csv',index=False)


In [115]:
x_samp = x_train_othr.sample(frac=0.01)
y_samp = y_train[x_samp.index]
x_test_samp = x_test_othr.sample(frac=0.01)